# Preparation

In [ ]:
!pip uninstall pyterrier

In [ ]:
!pip install pyterrier
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git
import pyterrier as pt
import pandas as pd
import csv
import os
import shutil

if not pt.started():
  pt.init()

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-req-build-wj6z49pl
  Running command git clone -q https://github.com/terrier-org/pyterrier.git /tmp/pip-req-build-wj6z49pl
     |████████████████████████████████| 81kB 3.2MB/s 
     |████████████████████████████████| 1.1MB 7.6MB/s 
     |████████████████████████████████| 71kB 4.7MB/s 
     |████████████████████████████████| 163kB 15.6MB/s 
     |████████████████████████████████| 51kB 3.5MB/s 
     |████████████████████████████████| 133kB 13.7MB/s 
     |████████████████████████████████| 1.8MB 13.9MB/s 
     |████████████████████████████████| 5.5MB 26.0MB/s 
     |████████████████████████████████| 645kB 25.5MB/s 
     |████████████████████████████████| 81kB 7.6MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.4.0-cp37-none-any.whl size=77384 sha256=652b830482d875cdb8fb4eef951702ddfbe9bb0648bd28c7e3c84091d516612e
  Stored in directory: /tmp/pip-ephem-wheel-cache-7aobbzyk/wheels/cc/bb/69/836d846a92c787b35

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Download data

In [ ]:
!wget -O /content/gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data/queries.tar.gz https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz
!wget -O /content/gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data/qrels.dev.tsv https://msmarco.blob.core.windows.net/msmarcoranking/qrels.dev.tsv
!wget -O /content/gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data/qrels.train.tsv https://msmarco.blob.core.windows.net/msmarcoranking/qrels.train.tsv
!wget -O /content/gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data/collection.tar.gz https://msmarco.blob.core.windows.net/msmarcoranking/collection.tar.gz
!wget -O /content/gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data19/2019qrels-pass.txt https://trec.nist.gov/data/deep/2019qrels-pass.txt
!wget -O /content/gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data19/queries.tar.gz https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz

--2021-02-24 17:19:22--  https://msmarco.blob.core.windows.net/msmarcoranking/queries.tar.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18882551 (18M) [application/gzip]
Saving to: ‘/content/gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data/queries.tar.gz’

/content/gdrive/MyD 100%[===================>]  18.01M  5.06MB/s    in 4.0s    

2021-02-24 17:19:26 (4.50 MB/s) - ‘/content/gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data/queries.tar.gz’ saved [18882551/18882551]

--2021-02-24 17:19:26--  https://msmarco.blob.core.windows.net/msmarcoranking/qrels.dev.tsv
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.


In [ ]:
cwd=os.getcwd()
print(cwd)

/content


# Unzip data

In [ ]:
import os
import tarfile

def unzip_all_data():
    cwd = os.getcwd()
    path = os.path.join(cwd, 'gdrive/MyDrive/Colab_Notebooks/Trec-DeepLearning-2020-master/data/')   # Change according to your path 

    os.chdir(path)

    # unzip queries

    with tarfile.open('queries.tar.gz', 'r:gz') as tar:
        tar.extractall()

    # unzip passages
    with tarfile.open('collection.tar.gz', 'r:gz') as tar:
        tar.extractall()


    path19 = os.path.join(cwd, 'data19')
    os.chdir(path19)

    with tarfile.open('queries.tar.gz', 'r:gz') as tar:
        tar.extractall()

    os.chdir(cwd)
    return None

# uncomment if it is the first time unzipping the data
unzip_all_data()


# Create Index

In [ ]:
import os
path ='/content/drive/My Drive/IR/'
os.chdir(path)

def passages_generator(filepath : str, delimiter : str, verbose : bool=False):
    csv_file = open(filepath)
    read_csv = csv.reader(csv_file, delimiter=delimiter)

    for i, (docno, text) in enumerate(read_csv):
        if i % 200000 == 0 and verbose:
            print(f'Processing passage {i}')
        yield {'docno': docno, 'text': text}

def create_index():
    """
    Creates pyterrier index using IterDictIndexer from pyterrier.
    """
    if os.path.exists('passage_index'):
        shutil.rmtree('passage_index')
    index_path = os.path.join(os.getcwd(),'passage_index')
    iter_indexer = pt.IterDictIndexer(index_path)
    collection_file = os.path.join(os.getcwd(),'data','collection.tsv')
    doc_iter = passages_generator(collection_file, '\t', verbose=True)
    index_passages = iter_indexer.index(doc_iter,meta=['docno', 'body'])
    print("done")
    return None
    
# uncomment to create index if index is not yet created.
create_index()

'''
dataset = pt.get_dataset("trec-deep-learning-passages")
def msmarco_generate():
    with pt.io.autoopen(dataset.get_corpus()[0], 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

iter_indexer = pt.IterDictIndexer("./passage_index")
indexref3 = iter_indexer.index(msmarco_generate(), meta=['docno', 'text'], meta_lengths=[20, 4096])
'''

Processing passage 0
Processing passage 200000
Processing passage 400000
22:19:47.922 [ForkJoinPool-1-worker-3] WARN  o.t.structures.indexing.Indexer - Adding an empty document to the index (500080) - further warnings are suppressed
Processing passage 600000
Processing passage 800000
Processing passage 1000000
Processing passage 1200000
Processing passage 1400000
Processing passage 1600000
Processing passage 1800000
Processing passage 2000000
Processing passage 2200000
Processing passage 2400000
Processing passage 2600000
Processing passage 2800000
Processing passage 3000000


# Connect to index, qrels, topics

In [ ]:
# paths
import os
print(os.getcwd())
cwd = os.getcwd()
#path = os.path.join(cwd, 'drive/My Drive/IR/data/')   # Change according to your path 
path ='/content/drive/My Drive/IR/data/'
os.chdir(path)
index_path = '/content/drive/My Drive/IR/pa_index/'
qrels_path= os.getcwd()
qrels_eval19_path = '/content/drive/My Drive/IR/data19/2019qrels-pass.txt'#os.path.join(os.getcwd(),'data19', '2019qrels-pass.txt')
qrels_train_path = os.path.join(qrels_path, 'qrels.train.tsv')
qrels_dev_path = os.path.join(qrels_path, 'qrels.dev.tsv')

topics_path = os.getcwd()#os.path.join(os.getcwd(),'data')
topics_train_path = os.path.join(topics_path, 'queries.train.tsv')
topics_random_train_path=os.path.join(topics_path,'Train.random1k.tsv')
topics_dev_path = os.path.join(topics_path, 'queries.dev.tsv')
topics_random_dev_path=os.path.join(topics_path,'Dev.random100.tsv')
#topics_eval_path = os.path.join(topics_path, 'queries.eval.tsv')
topics_eval19_path = '/content/drive/My Drive/IR/data19/queries.eval.tsv'#os.path.join(os.getcwd(),'data19', 'queries.eval.tsv')
topics_test_path=os.path.join(topics_path,'msmarco-test2019-queries.tsv')
# read data into dataframes from paths
#topics_train = pt.io.read_topics(topics_train_path, format='singleline')
topics_random_train=pt.io.read_topics(topics_random_train_path, format='singleline')
topics_dev = pt.io.read_topics(topics_dev_path, format='singleline')
topics_random_dev=pt.io.read_topics(topics_random_dev_path, format='singleline')
#topics_eval = pt.io.read_topics(topics_eval_path, format='singleline')
topics_eval19 = pt.io.read_topics(topics_eval19_path, format='singleline')
topics_test=pt.io.read_topics(topics_test_path,format='singleline')
qrels_train = pt.io.read_qrels(qrels_train_path)
qrels_dev = pt.io.read_qrels(qrels_dev_path)
qrels_eval19 = pt.io.read_qrels(qrels_eval19_path)

indexRef = pt.TRECCollectionIndexer(index_path)

/content


# Preprocess empty topics

some topics have empty queries for some reason, and that messes up pyterrier_bert so let's fix it

In [ ]:
def fill_empty_queries(df: pd.DataFrame):
    """
    fills all empty queries with some text so that pyterrier_bert does not crash.
    """
    df_copy = df.copy()
    df_copy.loc[df_copy['query'].str.len() == 0,'query'] = 'nova'
    return df_copy


topics_train = fill_empty_queries(topics_train)
topics_dev = fill_empty_queries(topics_dev)
#topics_eval = fill_empty_queries(topics_eval)
topics_test=fill_empty_queries(topics_test)
topics_eval19 = fill_empty_queries(topics_eval19)

# Experiments

Filtering training and dev data

In [ ]:
#Filtering training and dev data
qrels_train_path = os.path.join(qrels_path, 'qrels.train.tsv')
qrels_dev_path = os.path.join(qrels_path, 'qrels.dev.tsv')
queries_train = 'content/drive/My drive/IR/data/queries.train.tsv'
queries_dev = 'content/drive/My drive/IR/data/queries.dev.tsv'

f1 = open(qrels_train_path,"r")
que = set()
doc = set()
for l in f1:
  k = l.split('\t')
  que.add(k[0])
  doc.add(k[2])
fq = open(topics_train_path, "r", encoding='UTF-8')
out = open("queries_samlltrain.tsv", "w", encoding='UTF-8')
print(len(que))
print(len(doc))

for q in fq:
  k = q.split('\t')
  if(k[0] in que):
    s = k[0] + k[1]
    out.write(s)
out.close()

f1 = open(qrels_dev_path,"r")
que = set()
doc = set()
for l in f1:
  k = l.split('\t')
  que.add(k[0])
  doc.add(k[2])
print(len(que))
print(len(doc))
fq = open(topics_dev_path, "r", encoding='UTF-8')
out = open("queries_samlldev.tsv", "w", encoding='UTF-8')
for q in fq:
  k = q.split('\t')
  if(k[0] in que):
    s =  k[0] + k[1] 
    out.write(s)
out.close()

502939
516472
55578
59096


In [ ]:
import os
import pandas as pd
from google.colab import drive
import numpy as np
import random

drive.mount('/content/drive')
cwd=os.getcwd()
path ='/content/drive/My Drive/IR/data/'   # Change according to your path 
queries_id_list=[]
queries_topic_list=[]
queries_dic={}

train_path=os.path.join(path,'queries_samlltrain.tsv')
train_data=pd.read_csv(train_path,sep='\t')

#row_data=dev_data.iloc[10000]
#print(row_data)
#print(row_data[1])
#f=open(os.path.join(os.getcwd(),'DEV.random200.tsv'),'w')
#for i in f.readlines():
f=open(os.path.join(os.getcwd(),'Train.random1k.tsv'),'w')
random_train_path=os.path.join(os.getcwd(),'Train.random1k.tsv')
'''
for index, row in dev_data.iterrows():
#print(row)
  queries_dic[str(row[0])]=str(row[1])
#print(list)
'''
random_number=random.sample(range(0,502938),1000)
#print(random_number[0])
#for i in random_number:
accum=0
for i in random_number:
  random_choice=train_data.iloc[random_number[accum]]
  #print(random_choice.str.split('\t'))
  k = random_choice.str.split('\t')
  st=k[0]
  flag = True
  #print(st[0])
  d = st[0]
  #print(d[0])
  for n in range(len(d)-1):
    if not d[n+1].isdigit():
      #print(d[0:n])
      #print(d[n+1:-1])
      queries_id_list.append(d[0:n+1])
      queries_topic_list.append(d[n+1:])
      break
  accum+=1
print(queries_id_list)
print(queries_topic_list)
queries_dic['A']=queries_id_list
queries_dic['B']=queries_topic_list
#pd.DataFrame(columns=['A','B'], index=list(range(200)))
#print(queries_dic)
df=pd.DataFrame(queries_dic)
df.to_csv(random_train_path,index=False,header=False,sep='\t')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['443611', '473830', '50025', '751375', '897840', '436730', '687210', '754836', '807305', '320858', '1049890', '545871', '165032', '428620', '649161', '253640', '71653', '166316', '806217', '781189', '116856', '29361', '423632', '235538', '277028', '1063075', '269466', '540552', '443157', '244017', '124626', '650402', '852102', '963707', '268450', '1157703', '14246', '514581', '210743', '181158', '55571', '580546', '1141965', '745839', '92535', '720038', '569395', '1004904', '141189', '1059585', '1180907', '1047650', '553624', '89163', '77412', '23593', '977779', '714433', '762476', '543625', '278546', '1153890', '1165280', '344428', '230342', '437478', '33325', '1168341', '583164', '244699', '993586', '567673', '704633', '1145512', '241976', '814702', '395645', '1151676', '998950', '66203', '681990', '907415', '994140', '442281', '114480', '789090', '955057'

# Random sample around 200 queries from dev dataset


In [ ]:
#random sample around 200 queries from dev datset
import os
import pandas as pd
from google.colab import drive
import numpy as np
import random

drive.mount('/content/drive')
cwd=os.getcwd()
path ='/content/drive/My Drive/IR/data/'   # Change according to your path 
queries_id_list=[]
queries_topic_list=[]
queries_dic={}

train_path=os.path.join(path,'queries_samlldev.tsv')
#train_path=os.path.join(path,'queries.train.tsv')
train_data=pd.read_csv(train_path,sep='\t')

#row_data=dev_data.iloc[10000]
#print(row_data)
#print(row_data[1])
#f=open(os.path.join(os.getcwd(),'DEV.random200.tsv'),'w')
#for i in f.readlines():
f=open(os.path.join(os.getcwd(),'Dev.random100.tsv'),'w')
random_train_path=os.path.join(os.getcwd(),'Dev.random100.tsv')
'''
for index, row in dev_data.iterrows():
#print(row)
  queries_dic[str(row[0])]=str(row[1])
#print(list)
'''
random_number=random.sample(range(0,55577),100)
#print(random_number[0])
#for i in random_number:
accum=0
for i in random_number:
  random_choice=train_data.iloc[random_number[accum]]
  #print(random_choice.str.split('\t'))
  k = random_choice.str.split('\t')
  st=k[0]
  flag = True
  #print(st[0])
  d = st[0]
  #print(d[0])
  for n in range(len(d)-1):
    if not d[n+1].isdigit():
      #print(d[0:n])
      #print(d[n+1:-1])
      queries_id_list.append(d[0:n+1])
      queries_topic_list.append(d[n+1:])
      break
  accum+=1
print(queries_id_list)
print(queries_topic_list)
queries_dic['A']=queries_id_list
queries_dic['B']=queries_topic_list
#pd.DataFrame(columns=['A','B'], index=list(range(200)))
#print(queries_dic)
df=pd.DataFrame(queries_dic)
df.to_csv(random_train_path,index=False,header=False,sep='\t')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
['1094988', '638324', '662761', '987202', '1081974', '214944', '50244', '420982', '69689', '454765', '995546', '518675', '332953', '700571', '1101374', '1037724', '1091665', '391712', '534910', '985734', '57865', '632104', '1102275', '342748', '753123', '1010835', '53393', '17006', '928215', '273545', '279477', '106098', '1083524', '986770', '1087363', '131803', '825726', '1091807', '1096618', '1088387', '953562', '812491', '570930', '1100490', '875219', '649844', '191994', '957697', '1093384', '1041941', '1033223', '456529', '755901', '384796', '534385', '746365', '831292', '1085777', '980200', '186473', '1051372', '1028122', '642655', '127099', '1021515', '410309', '612253', '1096567', '79633', '912767', '1005314', '986071', '1101339', '1100975', '459393', '1087313', '1048600', '1048020', '621867', '735895', '351380', '1090034', '126329', '260404', '984429'

# L2R baseline 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

results_path = os.path.join(os.getcwd(), 'results')
rf = RandomForestRegressor(n_estimators=400)
BM25_br = pt.BatchRetrieve(indexRef, controls={"wmodel" : "BM25"}, verbose=True)
tf = pt.BatchRetrieve(indexRef,metadata=["docno","text"],controls={"wmodel" : "TF_IDF"}, verbose=True)
pl2 = pt.BatchRetrieve(indexRef,metadata=["docno","text"],controls={"wmodel" : "PL2"}, verbose=True)
DPH_br = pt.BatchRetrieve(indexRef,metadata=["docno","text"], wmodel="DPH",verbose=True) 
DLM = pt.BatchRetrieve(indexRef,metadata=["docno","text"], wmodel="DirichletLM",verbose=True)
pipeline=BM25_br>> (tf**pl2**BM25_br**DPH_br**DLM)

rf_pipe = pipeline >> pt.ltr.apply_learned_model(rf)
print(topics_random_dev)

rf_pipe.fit(topics_random_train,qrels_train)


# error analysis';'

print("--------------error analysis----------------------")
per_df_result_eval19 = pt.pipelines.Experiment([rf_pipe],
                        topics_random_dev,
                        qrels_dev,
                        ['map','ndcg'],  
                        names=["L2R baseline error analysis"],
                        perquery=True)

f=open(os.path.join(results_path,'error_baseline_retrieval__L2R_baseline_per.csv'),'w')
per_df_result_eval19.to_csv(
    os.path.join(results_path, 'error_baseline_retrieval__L2R_baseline_per.csv'), \
    index=False)

# baseline for 2019 test set
print("--------------per query eval19----------------------")
per_df_result_eval19 = pt.pipelines.Experiment([rf_pipe],
                        topics_test,
                        qrels_eval19,
                        ['official','ndcg'],  
                        names=["L2R baseline"],
                        perquery=True)
f=open(os.path.join(results_path,'eval19_baseline_retrieval__L2R_baseline_per.csv'),'w')
per_df_result_eval19.to_csv(
    os.path.join(results_path, 'eval19_baseline_retrieval__L2R_baseline_per.csv'), \
    index=False)
print("--------------overal query eval19----------------------")
df_result_eval19 = pt.pipelines.Experiment([rf_pipe],
                        topics_test,
                        qrels_eval19,
                        ['official','ndcg'],  
                        names=["L2R baseline"])
f=open(os.path.join(results_path,'eval19_baseline_retrieval__L2R_baseline_total.csv'),'w')
df_result_eval19.to_csv(
    os.path.join(results_path, 'eval19_baseline_retrieval__L2R_baseline_total.csv'), \
    index=False)


print('Finished eval 19 L2R baseline')

22:03:19.830 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
22:03:22.222 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
22:03:24.203 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
22:03:26.180 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
22:03:28.229 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta readi

BR(BM25):   0%|          | 0/1000 [00:00<?, ?q/s]

BR(TF_IDF):   0%|          | 0/1000 [00:00<?, ?q/s]

BR(PL2):   0%|          | 0/1000 [00:00<?, ?q/s]

BR(BM25):   0%|          | 0/1000 [00:00<?, ?q/s]

...

BR(PL2):   0%|          | 0/200 [00:00<?, ?q/s]

BR(BM25):   0%|          | 0/200 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/200 [00:00<?, ?q/s]

BR(DirichletLM):   0%|          | 0/200 [00:00<?, ?q/s]

Finished eval 19 L2R baseline


# Pairwise approach

In [ ]:
import xgboost as xgb

results_path = os.path.join(os.getcwd(), 'results')
BM25_br = pt.BatchRetrieve(indexRef, controls={"wmodel" : "BM25"}, verbose=True)
tf = pt.BatchRetrieve(indexRef, controls={"wmodel" : "Tf"}, verbose=True)
pl2 = pt.BatchRetrieve(indexRef, controls={"wmodel" : "PL2"}, verbose=True)
pipeline=BM25_br>> (tf ** pl2)
params = {'objective': 'rank:ndcg', 
          'learning_rate': 0.1, 
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6,
          'verbose': 2,
          'random_state': 42 
         }

BaseLTR_LM = pipeline >> pt.pipelines.XGBoostLTR_pipeline(xgb.sklearn.XGBRanker(**params))
BaseLTR_LM.fit(topics_random_train,qrels_train)
print("--------------error analysis----------------------")
per_df_result_eval19 = pt.pipelines.Experiment([BaseLTR_LM],
                        topics_random_dev,
                        qrels_dev,
                        ['map','ndcg'],  
                        names=["L2R pairwise error analysis"],
                        perquery=True)

f=open(os.path.join(results_path,'error_pairwise_retrieval__L2R_pairwise_per.csv'),'w')
per_df_result_eval19.to_csv(
    os.path.join(results_path, 'error_pairwise_retrieval__L2R_pairwise_per.csv'), \
    index=False)

# baseline for 2019 test set
print("--------------per query eval19----------------------")
per_df_result_eval19 = pt.pipelines.Experiment([BaseLTR_LM],
                        topics_test,
                        qrels_eval19,
                        ['official','ndcg'],  
                        names=["L2R pairwise"],
                        perquery=True)
f=open(os.path.join(results_path,'eval19p_pairwise_retrieval__L2R_pairwise_per.csv'),'w')
per_df_result_eval19.to_csv(
    os.path.join(results_path, 'eval19_pairwise_retrieval__L2R_pairwise_per.csv'), \
    index=False)
print("--------------overal query eval19----------------------")
df_result_eval19 = pt.pipelines.Experiment([BaseLTR_LM],
                        topics_test,
                        qrels_eval19,
                        ['official','ndcg'],  
                        names=["L2R pairwise"])
f=open(os.path.join(results_path,'eval19_pairwise_retrieval__L2R_pairwise_total.csv'),'w')
df_result_eval19.to_csv(
    os.path.join(results_path, 'eval19_pairwise_retrieval__L2R_pairwise_total.csv'), \
    index=False)


print('Finished eval 19 L2R pairwise')


20:54:09.784 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
20:54:12.117 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
20:54:14.260 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecatedWarning: XGBoostLTR_pipeline is deprecated as of 0.3.0. Please use pt.ltr.apply_learned_model(learner, form='ltr')
  app.launch_new_instance()
BR(PL2): 100%|██████████| 100/100 [00:11<00:00,  8.57q/s]


TypeError: ignored

In [ ]:
!pip install  --upgrade git+https://github.com/cmacdonald/pyterrier_bert.git

from pyterrier_bert.bert4ir import *
!pip install ir_datasets
import ir_datasets

dataset = ir_datasets.load('msmarco-passage/train/judged')
'''
for query in dataset.queries_iter():
    print(query)# namedtuple<query_id, text>
'''
for doc in dataset.docs_iter():
    print(doc) # namedtuple<doc_id, text>
results_path = os.path.join(os.getcwd(), 'results')
dataset = pt.get_dataset("irds:msmarco-passage/train/split200-valid")
topics = dataset.get_topics().head(100)
print(topics)
qrels = dataset.get_qrels()
BM25_br = pt.BatchRetrieve(indexRef,metadata=["docno","text"],controls={"wmodel" : "BM25"}, verbose=True)
bertpipe = BM25_br >> BERTPipeline()
print("-----------------bertpipe.fit-------------------------")
#bertpipe.fit(topics_random_train, qrels_train)
bertpipe.fit(topics, qrels)
# baseline for 2019 test set
print("--------------error analysis----------------------")
per_df_result_eval19 = pt.pipelines.Experiment([bertpipe],
                        topics_random_dev,
                        qrels_dev,
                        ['map','ndcg'],  
                        names=["BM25 + bert4ir error analysis"],
                        perquery=True)
per_df_result_eval19.to_csv(
    os.path.join(results_path, 'error_BM25bert4ir_retrieval_per.csv'), \
    index=False)
print("Finished error analysis")
print("--------------BM25 + bert4ir per query eval19----------------------")
per_df_result_eval19 = pt.pipelines.Experiment([bertpipe],
                        topics_test,
                        qrels_eval19,
                        ['official','ndcg'],  
                        names=["BM25 + bert4ir"],
                        perquery=True)

df_result_eval19.to_csv(
    os.path.join(results_path, 'eval19_BM25bert4ir_retrieval_per.csv'), \
    index=False)
print('Finished BM25 + bert4ir per query eval 19')
print("--------------BM25 + bert4ir total query eval19----------------------")
df_result_eval19=pt.pipelines.Experiment([bertpipe],
                        topics_test,
                        qrels_eval19,
                        ['official','ndcg'],
                        names=["BM25 + bert4ir"])
df_result_eval19.to_csv(
    os.path.join(results_path, 'eval19_BM25bert4ir_retrieval_total.csv'),\
    index=False
)
print("Finished BM25 + bert4ir total query eval")
'''
# baseline for 2020 eval

df_baseline_retrieval_2020 = bertpipe.transform(topics_eval)
df_baseline_retrieval_2020.to_csv(
    os.path.join(results_path, 'eval20_baseline_retrieval__bm25_bert4ir.csv'), \
    index=False)
print('Finished eval 20')
'''

  Cloning https://github.com/cmacdonald/pyterrier_bert.git to /tmp/pip-req-build-adl67dyt
  Running command git clone -q https://github.com/cmacdonald/pyterrier_bert.git /tmp/pip-req-build-adl67dyt
  Created wheel for PyTerrierBERT: filename=PyTerrierBERT-0.0.1-cp37-none-any.whl size=16832 sha256=e6d1ce2a083135ada6b03eef37932e10b5bb57e0313871131331a1db52e9040c
  Stored in directory: /tmp/pip-ephem-wheel-cache-984_myhn/wheels/12/4f/8f/a0aeba5949bd2bf556cd2e601b4dad684b6dc50dc759138fed
Successfully built PyTerrierBERT
  Found existing installation: PyTerrierBERT 0.0.1
    Uninstalling PyTerrierBERT-0.0.1:
      Successfully uninstalled PyTerrierBERT-0.0.1


流式输出内容被截断，只能显示最后 5000 行内容。


fixing encoding: 19.0MB [00:07, 2.62MB/s]



fixing encoding: 19.5MB [00:07, 2.64MB/s]



fixing encoding: 19.9MB [00:07, 2.67MB/s]



fixing encoding: 20.4MB [00:07, 2.69MB/s]



fixing encoding: 20.9MB [00:07, 2.72MB/s]



fixing encoding: 21.4MB [00:07, 2.74MB/s]



fixing encoding: 21.9MB [00:07, 2.77MB/s]



fixing encoding: 22.4MB [00:08, 2.79MB/s]



fixing encoding: 22.9MB [00:08, 2.81MB/s]



fixing encoding: 23.4MB [00:08, 2.83MB/s]



fixing encoding: 23.8MB [00:08, 2.86MB/s]



fixing encoding: 24.3MB [00:08, 2.88MB/s]



fixing encoding: 24.8MB [00:08, 2.90MB/s]



fixing encoding: 25.3MB [00:08, 2.92MB/s]



fixing encoding: 25.8MB [00:08, 2.94MB/s]



fixing encoding: 26.3MB [00:08, 2.96MB/s]



fixing encoding: 26.8MB [00:09, 2.97MB/s]



fixing encoding: 27.3MB [00:09, 2.99MB/s]



fixing encoding: 27.8MB [00:09, 3.01MB/s]



fixing encoding: 28.3MB [00:09, 3.03MB/s]



fixing encoding: 28.8MB [00:09, 3.04MB/s]



fixing encoding: 29.2MB [0

KeyboardInterrupt: ignored

In [ ]:
import pyterrier as pt

trecdl = pt.datasets.get_dataset("trec-deep-learning-docs")
# your index must have the contents of the documents recorded as metadata

def msmarco_generate():
    #dataset = pt.get_dataset("trec-deep-learning-docs")
    with pt.io.autoopen(trecdl.get_corpus()[0], 'rt') as corpusfile:
        for l in corpusfile:
            docno, url,title,body = l.split("\t")
            yield {'docno':docno,'url':url,'title':title,'body':body}

iter_indexer = pt.IterDictIndexer("/content/drive/My Drive/IR/doc_index")
indexref = iter_indexer.index(msmarco_generate(), meta=['docno','text','url','body'])
index = pt.IndexFactory.of(indexref)
# load the index, print the statistics
#index = pt.IndexFactory.of(indexref)

#print(index.getCollectionStatistics().toString())

qrelsTest = trecdl.get_qrels("test")
qrelsTrain = trecdl.get_qrels("train")
#take 1000 topics for training
topicsTrain = trecdl.get_topics("train").head(10)
#take 50 topics for validation
topicsValid = trecdl.get_topics("train").iloc[1001:1010]
#this one-liner removes topics from the test set that do not have relevant documents
topicsTest = trecdl.get_topics("test").merge(qrelsTest[qrelsTest["label"] > 0][["qid"]].drop_duplicates())

# initial retrieval and QE baseline.
#index = pt.IndexFactory.of(pt.IndexRef.of(indexloc))
DPH_br = pt.BatchRetrieve(index, controls={"wmodel" : "DPH"}, verbose=True)
DPH_br_qe = pt.BatchRetrieve(index, controls={"wmodel" : "DPH", "qe" : "on"}, verbose=True)

from pyterrier_bert.pyt_cedr import CEDRPipeline

cedrpipe = DPH_br >> CEDRPipeline(max_valid_rank=20)
# training, this uses validation set to apply early stopping
cedrpipe.fit(topicsTrain, qrelsTrain, topicsValid, qrelsTrain)

# testing performance
pt.pipelines.Experiment(topicsTest, 
                        [DPH_br, DPH_qe, cedrpipe],
                        ['map', 'ndcg'], 
                        qrelsTest, 
                        names=["DPH", "DPH + CEDR BERT"])


msmarco-docs.trec.gz: 100%|██████████| 7.92G/7.92G [32:04<00:00, 4.42MiB/s]
Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/pyterrier/index.py", line 582, in _write_fifos
    for doc in it:
  File "<ipython-input-3-7b0741177b5b>", line 10, in msmarco_generate
    docno, url,title,body = l.split("\t")
ValueError: not enough values to unpack (expected 4, got 1)



JavaException: ignored